In [1]:
import tensorflow as tf
import numpy as np
import math
import os
import random
import numpy as np
import pandas as pd
from pandas import DataFrame
import cPickle as pickle
import math

In [2]:
with open('../data/state_features.txt') as f:
    state_features = f.read().split()
print (state_features)
print len(state_features)

['Albumin', 'Arterial_BE', 'Arterial_lactate', 'Arterial_pH', 'BUN', 'CO2_mEqL', 'Calcium', 'Chloride', 'Creatinine', 'DiaBP', 'FiO2_1', 'GCS', 'Glucose', 'HCO3', 'HR', 'Hb', 'INR', 'Ionised_Ca', 'Magnesium', 'MeanBP', 'PT', 'PTT', 'PaO2_FiO2', 'Platelets_count', 'Potassium', 'RR', 'SGOT', 'SGPT', 'SIRS', 'SOFA', 'Shock_Index', 'Sodium', 'SpO2', 'SysBP', 'Temp_C', 'Total_bili', 'WBC_count', 'Weight_kg', 'age', 'elixhauser', 'gender', 'mechvent', 'output_4hourly', 'output_total', 'paCO2', 'paO2', 're_admission']
47


In [3]:
df = pd.read_csv('../data/rl_train_data_final_cont.csv')

In [4]:
df_test = pd.read_csv("../data/rl_test_data_final_cont.csv")

In [ ]:
df_val = pd.read_csv("../data/rl_val_data_final_cont.csv")

In [5]:
n_input = len(state_features)
n_hidden_1 = 200
weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b1': tf.Variable(tf.random_normal([n_input])),
}

In [6]:
class AutoEncoder:
    def __init__(self):
        self.input_size = len(state_features)
        self.batch_size = tf.placeholder(tf.int32)
        self.sparsity_dist = tf.fill([self.batch_size,n_hidden_1],0.05)
        self.X = tf.placeholder("float", [None,self.input_size])
        self.layer_1_a = tf.nn.sigmoid(tf.add(tf.matmul(self.X, weights['encoder_h1']),
                                   biases['encoder_b1']))
        self.layer_1_d = tf.nn.sigmoid(tf.add(tf.matmul(self.layer_1_a, weights['decoder_h1']),
                                   biases['decoder_b1']))
        self.target_y = self.X
        self.kl = tf.reduce_mean(tf.reduce_sum(tf.multiply(self.sparsity_dist,tf.log(self.sparsity_dist/self.layer_1_a)) + \
                                                tf.multiply((1-self.sparsity_dist),tf.log((1-self.sparsity_dist)/(1-self.layer_1_a))), axis =1))
        # regularisation term logic - loss at the start is roughly 60, of this 0.5 is reconstruction loss.  
        self.loss = tf.reduce_mean(tf.pow(self.target_y - self.layer_1_d, 2)) + 0.0001*self.kl
        self.optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(self.loss)

In [7]:
df.keys()

Index([u'Unnamed: 0', u'Unnamed: 0.1', u'Unnamed: 0.1.1', u'bloc',
       u'icustayid', u'charttime', u'gender', u'age', u'elixhauser',
       u're_admission', u'died_in_hosp', u'mortality_90d', u'Weight_kg',
       u'GCS', u'HR', u'SysBP', u'MeanBP', u'DiaBP', u'Shock_Index', u'RR',
       u'SpO2', u'Temp_C', u'FiO2_1', u'Potassium', u'Sodium', u'Chloride',
       u'Glucose', u'BUN', u'Creatinine', u'Magnesium', u'Calcium',
       u'Ionised_Ca', u'CO2_mEqL', u'SGOT', u'SGPT', u'Total_bili', u'Albumin',
       u'Hb', u'WBC_count', u'Platelets_count', u'PTT', u'PT', u'INR',
       u'Arterial_pH', u'paO2', u'paCO2', u'Arterial_BE', u'Arterial_lactate',
       u'HCO3', u'PaO2_FiO2', u'mechvent', u'SOFA', u'SIRS',
       u'median_dose_vaso', u'max_dose_vaso', u'input_total_tev',
       u'input_4hourly_tev', u'output_total', u'output_4hourly',
       u'cumulated_balance_tev', u'vaso_input', u'iv_input', u'reward'],
      dtype='object')

In [8]:
def eval_on_val():
    cur_state = df_val.ix[:,state_features]
    l = sess.run([autoencoder.loss], feed_dict={autoencoder.X:cur_state, autoencoder.batch_size:len(df_val)})
    print "validation loss is: ", l

In [9]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [10]:
batch_size = 100
autoencoder = AutoEncoder()
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for step in range(100000):
        sum_loss = 0
        a = df.sample(n=batch_size)
        cur_state = a.ix[:,state_features]
        _,l = sess.run([autoencoder.optimizer, autoencoder.loss], feed_dict={autoencoder.X:cur_state, autoencoder.batch_size : batch_size})
        sum_loss += l
        if step % 1000==0 and step > 0:
            print "step is ", step 
            print "total loss is ", sum_loss
            sum_loss = 0
        if step % 20000 == 0 and step >0:
            b = df.sample(n=1)
            cur_state_trial = b.ix[:,state_features]
            target_y,reconstruction = sess.run([autoencoder.target_y,autoencoder.layer_1_d],feed_dict={autoencoder.X:cur_state_trial})
            print reconstruction
            print target_y
        if step % 20000 == 0 and step >0:
            eval_on_val()
    # run the model on the train, val, and test data to get a reduced dimensionality representation
    train_states = df.ix[:,state_features]
    train_repr = sess.run([autoencoder.layer_1_a], feed_dict={autoencoder.X:train_states})
    train_autoencode = pd.DataFrame(train_repr[0])
    train_autoencode['vaso_input'] = df['vaso_input']
    train_autoencode['iv_input'] = df['iv_input']
    train_autoencode['reward'] = df['reward']
    train_autoencode['icustayid'] = df['icustayid']
    
    val_states = df_val.ix[:,state_features]
    val_repr = sess.run([autoencoder.layer_1_a], feed_dict={autoencoder.X:val_states})
    val_autoencode = pd.DataFrame(val_repr[0])
    val_autoencode['vaso_input'] = df_val['vaso_input']
    val_autoencode['iv_input'] = df_val['iv_input']
    val_autoencode['reward'] = df_val['reward']
    val_autoencode['icustayid'] = df_val['icustayid']
    
    test_states = df_test.ix[:,state_features]
    test_repr = sess.run([autoencoder.layer_1_a], feed_dict={autoencoder.X:test_states})
    test_autoencode = pd.DataFrame(test_repr[0])
    test_autoencode['vaso_input'] = df_test['vaso_input']
    test_autoencode['iv_input'] = df_test['iv_input']
    test_autoencode['reward'] = df_test['reward']
    test_autoencode['icustayid'] = df_test['icustayid']

step is  1000
total loss is  0.104755602777
step is  2000
total loss is  0.0705829635262
step is  3000
total loss is  0.0535706803203
step is  4000
total loss is  0.0427041202784
step is  5000
total loss is  0.0386305488646
step is  6000
total loss is  0.0305601842701
step is  7000
total loss is  0.0278552416712
step is  8000
total loss is  0.0268832594156
step is  9000
total loss is  0.0216995552182
step is  10000
total loss is  0.0192614011467
step is  11000
total loss is  0.0186603888869
step is  12000
total loss is  0.0157892331481
step is  13000
total loss is  0.0133222192526
step is  14000
total loss is  0.0112559879199
step is  15000
total loss is  0.0102339256555
step is  16000
total loss is  0.00958498008549
step is  17000
total loss is  0.00788414571434
step is  18000
total loss is  0.00702556129545
step is  19000
total loss is  0.00640633236617
step is  20000
total loss is  0.00622528325766
[[ 0.44204974  0.4467707   0.08276776  0.66697174  0.59531093  0.15064354
   0.518703

In [11]:
train_autoencode.to_csv("../data/rl_train_data_final_auto.csv", index=False)
val_autoencode.to_csv("../data/rl_val_data_final_auto.csv", index=False)
test_autoencode.to_csv("../data/rl_val_data_final_auto.csv", index=False)